# Comment analysis
- Some search doesn't match properly: Praça Bartolomeu de Gusmão

# Objectives
1. Explore downloaded data
2. Data cleaning
3. Analysis of score of every place
4. Words analysis
5. Spatial distribution of scoring

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
import sqlite3
import os
import re
from datetime import datetime
import geopandas as gpd
import osmnx as ox
import folium
import numpy as np


from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import fcluster 
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn import preprocessing

In [ ]:
print(os.getcwd(), os.listdir('./'))
os.chdir('./../')
print(os.getcwd(), os.listdir('./'))


In [ ]:
NOME_DB = "data/raw/scrapcomments.db"
with sqlite3.connect(NOME_DB) as connection:
    df = pd.read_sql('SELECT * FROM googleplaces', connection)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# 1598 - 801 = 797 Duplicates
df = df.drop_duplicates(subset=['url', 'Comentario'])
df.shape

# Explore data

In [ ]:
df['Estrellas'].value_counts()

In [ ]:
df.dtypes

In [ ]:
df['url']

In [ ]:
df['url'].iloc[0]

# Time series analysis

In [ ]:
df['Fecha'].value_counts()

In [ ]:
# Mapping words
MAPA_TIEMPO = {
    'years': ['ano', 'anos', 'año', 'años'],
    'months': ['mês', 'meses', 'mes'],
    'weeks': ['semana', 'semanas'],
    'days':   ['dia', 'dias', 'día', 'días'],
    'hours':  ['hora', 'horas'],  # Extra: por si aparecen horas
}

def convertir_fecha_dinamica(texto):
    """
    Convierte texto como 'há 6 anos' en un objeto datetime real
    usando extracción dinámica de números y unidades.
    """
    # 1. Normalización básica
    if pd.isna(texto): return pd.NaT
    # Pasamos a minúsculas y limpiamos espacios
    t = str(texto).lower().strip()
    
    # Fecha base (Hoy) normalizada a las 00:00:00
    hoy = pd.Timestamp.now().normalize()

    # 2. Extracción del VALOR (Entero)
    cantidad = 0
    
    # Caso especial: palabras textuales como "um" o "uma"
    if 'um ' in t or 'uma ' in t: # Espacio para no confundir con 'algUMa'
        cantidad = 1
    else:
        # Regex: Busca cualquier secuencia de dígitos (\d+)
        coincidencia = re.search(r'(\d+)', t)
        if coincidencia:
            cantidad = int(coincidencia.group(1))
        else:
            return pd.NaT # Si no hay número ni 'um', no podemos calcular

    # 3. Detección de la UNIDAD (Dinámica)
    # Recorremos nuestro diccionario de configuración
    for unidad_pandas, palabras_clave in MAPA_TIEMPO.items():
        # Verificamos si alguna palabra clave está en el texto
        if any(palabra in t for palabra in palabras_clave):
            
            # 4. Cálculo Matemático (Magia de Pandas)
            # Creamos un diccionario dinámico con el parámetro correcto
            # Equivale a decir: DateOffset(years=6) o DateOffset(months=3)
            kwargs = {unidad_pandas: cantidad}
            return hoy - pd.DateOffset(**kwargs)

    # Si llegamos aquí, detectamos número pero no la unidad (ej: "há 6 ???")
    return pd.NaT


In [ ]:
# Apply custom function to extract time data
print("Calculando fechas...")
df['Fecha_Calculada'] = df['Fecha'].apply(convertir_fecha_dinamica)
df.head()

# Simple exploratory comments
## Word Cloud

In [ ]:
# Descargamos las palabras vacías (solo la primera vez)
nltk.download('stopwords')

# Convertimos a string por si hay algún dato numérico suelto y eliminamos vacíos
df['Comentario'] = df['Comentario'].astype(str).fillna('')

# 2. DEFINIR STOPWORDS (Palabras a ignorar)
# Usamos la lista de español de NLTK y añadimos algunas propias que no aportan valor
stop_words_es = stopwords.words('portuguese')
# Añadimos palabras "basura" típicas de reviews que ensucian el gráfico
nuevas_stopwords = ['lar', 'sitio', 'ir', 'ver', 'mas', 'si', 'tan', 'parco', 'verde'] 
stop_words_es.extend(nuevas_stopwords)

In [ ]:
def generar_nube(texto_completo, titulo):
    wordcloud = WordCloud(
        width=800, 
        height=400,
        background_color='white',
        stopwords=stop_words_es,
        min_font_size=10
    ).generate(texto_completo)

    plt.figure(figsize=(10, 5), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(titulo, fontsize=20)
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
print("Generando Nube de Palabras General...")
texto_general = " ".join(review for review in df.Comentario)
generar_nube(texto_general, "Palabras más usadas en Parco Verde")

## B-Gram

In [ ]:
def grafico_bigramas(corpus, n=10):
    # CountVectorizer counting pairs of words (ngram_range=(2,2))
    vec = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words_es).fit(corpus)
    bag_of_words = vec.transform(corpus)
    
    # Sum of the word frequencies 
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    # Order frequency
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    # Get first n pair
    top_words = words_freq[:n]
    
    # Separamos para graficar
    x, y = zip(*top_words) # x=words, y=frequency
    
    plt.figure(figsize=(10, 6))
    plt.barh(x, y, color='skyblue')
    plt.gca().invert_yaxis()
    plt.title('Top 10 Pares de Palabras (Bigramas) más comunes')
    plt.xlabel('Frecuencia')
    plt.show()

In [ ]:
print("Generando Bigramas...")
grafico_bigramas(df['Comentario'], n=15)

# Time series

In [ ]:
df.head()

In [ ]:
df['Estrellas'] = df['Estrellas'].astype(int)

In [ ]:
analisis_temporal = df.groupby(pd.Grouper(key='Fecha_Calculada', freq='M')).agg({'Estrellas': 'mean'})
print(analisis_temporal)

In [ ]:
# Plot of stars over the time column
analisis_temporal['Estrellas'].plot(kind='line', marker='o', color='orange', figsize=(10,5))

plt.title("Evolución de la Calidad (Estrellas) por Mes")
plt.ylabel("Estrellas (1-5)")
plt.xlabel("Fecha")
plt.grid(True)
plt.show()

# Spatial comments analysis

In [ ]:
df.head()

In [ ]:
df['url'].iloc[0]

In [ ]:
# Extract coordinates
# (?P<Latitud>...) assign a new column
patron = r'@(?P<Latitud>-?\d+\.\d+),(?P<Longitud>-?\d+\.\d+)'

# 3. Aplicamos la extracción mágica
# Esto busca el patrón en cada fila y separa los grupos en nuevas columnas
coordenadas = df['url'].str.extract(patron)

# 4. Convertimos a números (porque se extraen como texto)
coordenadas = coordenadas.astype(float)

# 5. Unimos las nuevas columnas a tu DataFrame original
df_final = pd.concat([df, coordenadas], axis=1)
df_final.head()

In [ ]:
analisis_temporal = df_final.groupby(by='url').agg({'Estrellas': 'mean',
                                              'Latitud': 'max',
                                              'Longitud': 'max',
                                              'search_word': 'max'})

In [ ]:
analisis_temporal.head()

In [ ]:
x_feat, y_feat = 'Longitud', 'Latitud'
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax.scatter(analisis_temporal[x_feat], analisis_temporal[y_feat], s=2)
plt.ylabel(y_feat)
plt.xlabel(x_feat)
fig.tight_layout()
plt.show()

In [ ]:
file_name = "data/raw/espacios_verdes_coimbra.geojson"
gdf_parques = gpd.read_file(file_name)
gdf = gdf_parques.loc[gdf_parques['name'].notna(), :].copy()
gdf['name'].str.lower().to_list()
lon, lat = np.mean(gdf_parques["geometry"].centroid.x), np.mean(gdf_parques["geometry"].centroid.y)

In [ ]:
# @tag:workspaceTrust
m = folium.Map(location=[lat, lon], zoom_start=10, tiles='OpenStreetMap')

# --- PARTE A: TU CÓDIGO (POLÍGONOS / BARRIOS) ---
# Iteramos sobre el GeoDataFrame (gdf)
for _, r in gdf.iterrows():
    # Simplificamos la geometría para que el mapa cargue rápido
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    
    # Creamos la capa GeoJson
    geo_j_layer = folium.GeoJson(
        data=geo_j, 
        style_function=lambda x: {"fillColor": "orange", "color": "orange", "weight": 1, "fillOpacity": 0.3}
    )
    
    # Añadimos popup con el nombre de la zona
    folium.Popup(r["name"]).add_to(geo_j_layer)
    geo_j_layer.add_to(m)

# --- PARTE B: NUEVO CÓDIGO (PUNTOS DE COORDENADAS) ---
# Iteramos sobre el DataFrame de coordenadas (df)
for index, row in analisis_temporal.iterrows():
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        tooltip=row['search_word'], # Aparece al pasar el ratón (hover)
        popup=folium.Popup(f"<b>{row['search_word']} ,  {row['Estrellas']}</b>", max_width=300), # Aparece al hacer clic
        icon=folium.Icon(color="blue", icon="info-sign") # Icono azul
    ).add_to(m)
m

In [ ]:
# Layer polygon
gdf.head(2)

In [ ]:
# Plot park polygons and locations with gradient rampmap color
x_feat, y_feat, size = 'Longitud', 'Latitud', 'Estrellas'
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax = gdf.plot(facecolor="#B01F1F", alpha=0.5, linewidth=0.01, hatch="//", ax=ax)
ax = ax.scatter(analisis_temporal[x_feat], analisis_temporal[y_feat], s=(analisis_temporal[size]+30), c=analisis_temporal[size])
plt.ylabel(y_feat)
plt.xlabel(x_feat)
cbar = plt.colorbar(ax)
cbar.set_label(size)
fig.tight_layout()
plt.show()

In [ ]:
# Save data in processed folder
analisis_temporal.loc[:, ['search_word', 'Estrellas', 'Longitud', 'Latitud']].to_csv('data/processed/parcoverde.csv', index=False)

# Unsupervised learning
Hierarchical Clustering
https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/

In [ ]:
def agrupacion_jerarquica(df, ls_cols, nombre_barrio, dist=3, method='ward',
                          metric='euclidean', optimal_ordering=False):
    mosaicstr="""
    ab
    ab
    cb
    """
    fig, ax = plt.subplot_mosaic(mosaic=mosaicstr, figsize=(10, 6))
    # relacion entre pares de instancias, distancia y acumulado
    # Metodo aglomerativo, no divisivo
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html
    Z = linkage(df.loc[:, ls_cols].values, method, 
                metric, optimal_ordering=optimal_ordering)
    kden = dendrogram(Z, ax=ax['a'], color_threshold=dist)
    ax['a'].axhline(y=dist, color='r', linestyle='--')
    ax['a'].set_title(f"Dendograma - método {method}")
    ax['a'].set(xlabel='Parco', ylabel='Distancia')
    agglo_clusters = fcluster(Z, t=dist,criterion='distance')
    print("Nº Clusters: ", np.unique(agglo_clusters).shape[0])
    ax['b'].set_title(f"Clustering distribution k={np.unique(agglo_clusters).shape[0]}")
    ax['b'].set(xlabel=ls_cols[0], ylabel=ls_cols[1])
    axb = ax['b'].scatter(df[ls_cols[0]], df[ls_cols[1]], 
                          c=agglo_clusters, cmap='Dark2', s=20)

    ax['b'].legend()
    cbar = plt.colorbar(axb)
    cbar.set_label('k-clusters')
    ax['c'].plot(Z[:, 2])
    ax['c'].axhline(y=dist, color='r', linestyle='--')
    # ax['c'].set_title(f"Distancia en la aglomeración")
    ax['c'].set(xlabel='Nº parcos acumulados', ylabel='Distancia')
    fig.tight_layout()
    plt.show()
    return kden

In [ ]:
ag = agrupacion_jerarquica(df=analisis_temporal, nombre_barrio='search_word', 
                           dist=0.01, ls_cols=['Longitud', 'Latitud'], 
                           method='ward', metric='euclidean', 
                           optimal_ordering=True)

In [ ]:
# Scaling data
lscol = ['Longitud', 'Latitud', 'Estrellas']
min_max_scaler = preprocessing.MinMaxScaler()
minmx = min_max_scaler.fit(analisis_temporal.loc[:, lscol])
df2cluster = pd.DataFrame(minmx.transform(analisis_temporal.loc[:, lscol]), columns=lscol)

In [ ]:
ag = agrupacion_jerarquica(df=df2cluster, nombre_barrio='search_word', 
                           dist=1.2, ls_cols=['Longitud', 'Latitud', 'Estrellas'], 
                           method='ward', metric='euclidean', 
                           optimal_ordering=True)

# END